# Example of Extraction Transformation & Loading manually in PGVector using all classes

In this notebook, we will see how to retrieve data on movies released this week by extracting them from [sens-critique](https://www.senscritique.com/). We will transform them and do the embedding of the reviews with the LLM [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large/tree/main) and [Text Embedding Inference](https://github.com/huggingface/text-embeddings-inference). Finally, we will integrate this data into [PGVector Database](https://github.com/pgvector/pgvector)

The classes used in this notebook are those of the project where you are located, available [here](https://github.com/ilanaliouchouche/senscritique-weeklyreal-database/tree/main/etl)

<p align="left">
  <img src="res/sc.jpg" width="200">
</p>

## Setting up PGVector and TEI Containers

To set up the PGVector and TEI containers, follow the steps below:

<img src="res/hf.png" width="175">
<img src="res/pg.png" width="175">

1. Run the following command to start the PGVector container:

In [3]:
!docker run --name db_sc -p 5432:5432 -e POSTGRES_PASSWORD=dw2 -v data:/var/lib/postgresql/data  -d ankane/pgvector 

a8a7ba6f50f22fb0977d4b309ce854dce415085dada1bca11eee975e2715fa30


2. Run the following command to start the TEI container:

In [4]:
!docker run --name tei_sc -p 8088:80 -v llm_data:/data --pull always -d ghcr.io/huggingface/text-embeddings-inference:cpu-0.6 --model-id intfloat/multilingual-e5-large --revision refs/pr/5 

5aac3e7af10c7729dc0ca6d32eb5c027c086c16cbb6298bba7e54bdee88d1fdb


cpu-0.6: Pulling from huggingface/text-embeddings-inference
1f7ce2fa46ab: Pulling fs layer
fa13aa30fca7: Pulling fs layer
1a69b05b9ce5: Pulling fs layer
1a41f0382383: Pulling fs layer
55e34e6c563d: Pulling fs layer
34d5bc9d302a: Pulling fs layer
4668085708f1: Pulling fs layer
54b7b6a88445: Pulling fs layer
e26bddd52764: Pulling fs layer
6aac8003b927: Pulling fs layer
f0c18d3f2754: Pulling fs layer
85a0f22f938d: Pulling fs layer
429a11bfc213: Pulling fs layer
55e34e6c563d: Waiting
1a41f0382383: Waiting
34d5bc9d302a: Waiting
4668085708f1: Waiting
54b7b6a88445: Waiting
e26bddd52764: Waiting
6aac8003b927: Waiting
f0c18d3f2754: Waiting
85a0f22f938d: Waiting
429a11bfc213: Waiting
1a69b05b9ce5: Download complete
1a41f0382383: Verifying Checksum
1a41f0382383: Download complete
1f7ce2fa46ab: Verifying Checksum
1f7ce2fa46ab: Download complete
34d5bc9d302a: Verifying Checksum
34d5bc9d302a: Download complete
4668085708f1: Verifying Checksum
4668085708f1: Download complete
55e34e6c563d: Verifying C

## Necessary Imports

In [2]:
import warnings
warnings.filterwarnings('ignore')
from etl.extract import CurrentMovieExtractor
from etl.transform import FilmTransformer
from etl.load import FilmLoader
import os
from setup_vcb import SetupPGVector

# Environment Variables Definition
To set up the environment variables, use the following code:

In [3]:
# PGVector environment variables
os.environ['PG_HOSTNAME'] = 'localhost'
os.environ['PG_USERNAME'] = 'postgres'
os.environ['PG_HPORT'] = '5432'
os.environ['PG_CPORT'] = '5432'
os.environ['PG_DBNAME'] = 'postgres'
os.environ['PG_DBPASSWORD'] = 'dw2'
os.environ['PG_DATA'] = 'data'

# TEI environment variables
os.environ['TEI_HOSTNAME'] = 'localhost'
os.environ['TEI_DATA'] = 'llm_data'
os.environ['TEI_HPORT'] = '8088'
os.environ['TEI_CPORT'] = '80'
os.environ['TEI_MODEL'] = 'intfloat/multilingual-e5-large'
os.environ['TEI_REVISION'] = 'refs/pr/5'

In [4]:
# Check PGVector environment variables
pg_hostname = os.environ.get('PG_HOSTNAME')
pg_username = os.environ.get('PG_USERNAME')
pg_hport = os.environ.get('PG_HPORT')
pg_cport = os.environ.get('PG_CPORT')
pg_dbname = os.environ.get('PG_DBNAME')
pg_dbpassword = os.environ.get('PG_DBPASSWORD')
pg_data = os.environ.get('PG_DATA')

print(f"PGVector environment variables:")
print(f"PG_HOSTNAME: {pg_hostname}")
print(f"PG_USERNAME: {pg_username}")
print(f"PG_HPORT: {pg_hport}")
print(f"PG_CPORT: {pg_cport}")
print(f"PG_DBNAME: {pg_dbname}")
print(f"PG_DBPASSWORD: {pg_dbpassword}")
print(f"PG_DATA: {pg_data}")

# Check TEI environment variables
tei_hostname = os.environ.get('TEI_HOSTNAME')
tei_data = os.environ.get('TEI_DATA')
tei_hport = os.environ.get('TEI_HPORT')
tei_cport = os.environ.get('TEI_CPORT')
tei_model = os.environ.get('TEI_MODEL')
tei_revision = os.environ.get('TEI_REVISION')

print(f"\nTEI environment variables:")
print(f"TEI_HOSTNAME: {tei_hostname}")
print(f"TEI_DATA: {tei_data}")
print(f"TEI_HPORT: {tei_hport}")
print(f"TEI_CPORT: {tei_cport}")
print(f"TEI_MODEL: {tei_model}")
print(f"TEI_REVISION: {tei_revision}")


PGVector environment variables:
PG_HOSTNAME: localhost
PG_USERNAME: postgres
PG_HPORT: 5432
PG_CPORT: 5432
PG_DBNAME: postgres
PG_DBPASSWORD: dw2
PG_DATA: data

TEI environment variables:
TEI_HOSTNAME: localhost
TEI_DATA: llm_data
TEI_HPORT: 8088
TEI_CPORT: 80
TEI_MODEL: intfloat/multilingual-e5-large
TEI_REVISION: refs/pr/5


## Extraction of the films of the week with `CurrentMovieExtractor`

1. In this step, we will use the `CurrentMovieExtractor` class with the `extract_all_film_links` method to extract the films of the week


In [8]:
extractor = CurrentMovieExtractor()
extractor.extract_all_film_links()

Extracting all films links...


Done with all films links


In [9]:
print(f"{len(extractor.urls_films)} film links extracted for this week")

27 film links extracted for this week


2. Next, we will retrieve the details of the films using the `extract_all_film_data` method.


In [10]:
extractor.extract_all_film_data()

Extracting all films informations...


100%|██████████| 27/27 [39:15<00:00, 87.23s/it]


Done with all films, extracting reviews...


100%|██████████| 27/27 [05:05<00:00, 11.30s/it]

Done with all reviews


Normally, the longest step is behind us, it should take about 15 minutes

## Data Transformation with the FilmTransformer Class

To transform the extracted film data, we will use the `FilmTransformer` class. This class provides methods to clean and preprocess the data before loading it into the database. So, the transformation of numerical and categorical data and the embedding with TEI of the reviews."

Here's an example of how to use the `FilmTransformer` class:

In [20]:
transformer = FilmTransformer(extractor)

Computing embeddings...


100%|██████████| 192/192 [07:50<00:00,  2.45s/it]

Done with embeddings


We can now visualize the data in the form of a dataframe, let's display for example the reviews:

In [21]:
transformer.df_reviews

,film,is_negative,title,likes,comments,content,url,embedding
0,Shin Godzilla,False,Le colosse s'érode,100,49,"Regarder la série des Godzilla, c’est admirer ...",https://www.senscritique.com/film/shin_godzill...,"[-0.01608497, -0.006072742, 0.0033162965, -0.0..."
1,Shin Godzilla,False,The Legend of Godzilla - A Link to the Past,63,37,[AVANT-PROPOS/AVERTISSEMENT : Voir ce film le ...,https://www.senscritique.com/film/shin_godzill...,"[-0.004714112, -0.019787442, -0.02954265, -0.0..."
2,Shin Godzilla,False,Godzilla contre les politiciens,25,8,Le dernier Godzilla japonais produit par la To...,https://www.senscritique.com/film/shin_godzill...,"[0.0077194446, -0.009460152, -0.0062741246, -0..."
3,Shin Godzilla,False,Premier impact,19,2,Ce qui frappe dès le début c'est la marque de ...,https://www.senscritique.com/film/shin_godzill...,"[0.010712562, -0.044431668, -0.006084309, -0.0..."
4,Shin Godzilla,False,Gojira's Rising,15,6,"Après la version correcte de Gareth Edwards, v...",https://www.senscritique.com/film/shin_godzill...,"[0.024141837, -0.03297161, -0.003320909, -0.06..."
...,...,...,...,...,...,...,...,...
187,Role Play,True,Role Play par Asoliloque,0,0,En essayant de performer sur tous les registre...,https://www.senscritique.com/film/role_play/cr...,"[0.003610156, 0.0004173794, -0.027124945, -0.0..."
188,Captain Miller,False,Captain Miller par Mathieu Ash,2,0,"Un jeune homme en colère, des colons anglais b...",https://www.senscritique.com/film/captain_mill...,"[0.025426337, -0.017913302, -0.020367501, -0.0..."
189,Captain Miller,True,Captain Miller par Mathieu Ash,2,0,"Un jeune homme en colère, des colons anglais b...",https://www.senscritique.com/film/captain_mill...,"[0.025426337, -0.017913302, -0.020367501, -0.0..."
190,La Dernière Danse,False,un thriller psychologique intense,0,0,"""La Dernière danse de Sabry Jarod"" est un film...",https://www.senscritique.com/film/la_derniere_...,"[0.026355425, 0.0076752915, -0.027532866, -0.0..."


We see several pieces of information such as likes, content, and their embedding done by the LLM

# `SetupPGVector` class to create the PGVector database schema. 

This class provides a method to set up the necessary tables and indexes for the ETL process.

Here's an example of how to use the `SetupPGVector` class:

In [22]:
# If the database schema does not exist, create it

setup = SetupPGVector(dbname=os.getenv("PG_DBNAME"), user=os.getenv("PG_USER"), password=os.getenv("PG_DBPASSWORD"), host=os.getenv("PG_HOSTNAME"), port=os.getenv("PG_HPORT"))
setup.setup_vdb()

# Load data into the Vector Data Base with the `FilmLoader` class

This class allows connecting to the VectorDataBase through the psycopg2 driver and integrating documents using the loading method

In [23]:
loader = FilmLoader(transformer, dbname=os.getenv("PG_DBNAME"), user=os.getenv("PG_USER"), password=os.getenv("PG_DBPASSWORD"), host=os.getenv("PG_HOSTNAME"), port=os.getenv("PG_HPORT"))
loader.loading()

Connecting to database: dbname=postgres user=postgres password=dw2 host=localhost port=5432
Connected to database
Loading data...


Done with loading


# ETL Done

At this stage, the ETL is performed manually in a notebook for those who particularly appreciate it. To consult the data, simply go to the vector database as follows:

```bash 
docker exec -it db_sc psql -U postgres
```
And then in psql client make a query 
```bash
SELECT embedding FROM reviews;
```
And you will be able to see something like that:
<p align="left">
  <img src="res/emb.png" >
</p>